In [1]:
from mypulp import * 
import itertools

# 例題

In [6]:
model = Model('example')

# 変数
x1 = model.addVar(vtype='C', name='x1')  # vtypeはデフォルトで'C'なので省略可
x2 = model.addVar(name='x2')
x3 = model.addVar(ub=30.0, name='x3')
model.update()

# 制約式
model.addConstr(2*x1 + x2 + x3 <= 60)
model.addConstr(x1 + 2*x2 + x3 <= 60)

# 目的関数
model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)

# 最適化実行
model.optimize()
print('Optimal value: ', model.ObjVal)
for v in model.getVars():
    print(v.VarName, v.X)

Optimal value:  1230.0
x1 10.0
x2 10.0
x3 30.0


# 輸送問題

## 1品種

In [11]:
# 定数
d = [80, 270, 250, 160, 180]  # 需要
M = [500, 500, 500]  # 生産量の上限
c = [[4, 6, 9], [5, 4, 7], [6, 3, 4], [8, 5, 3], [10, 8, 4]]  # コスト
I = range(len(d))  # 顧客の集合
J = range(len(M))  # 工場の集合

model = Model('transportation')

# 変数
x = {}
for i in I:
    for j in J:
        x[i, j] = model.addVar(vtype='C', name=f'x{i}{j}', lb=0)
model.update()

# 定式化
for i in I:
    model.addConstr(quicksum(x[i, j] for j in J) == d[i], name=f'Demand({i})')
for j in J:
    model.addConstr(quicksum(x[i, j] for i in I) <= M[j], name=f'Capacity({j})')
model.setObjective(quicksum(c[i][j] * x[i, j] for i in I for j in J), GRB.MINIMIZE)

# 最適化実行
model.optimize()
print('Optimal value: ', model.ObjVal)
eps = 1.0e-6
for i, j in itertools.product(I, J):
    if x[i, j].X > eps:
        print(f'sending quantity: {x[i, j].X} (from factory {j} to customer {i})')
        
# 双対
print('\n**********************')
print('constants: slack, dual')
for c in model.getConstrs():
    print(f'{c.ConstrName}: {c.Slack}, {c.Pi}')

Optimal value:  3370.0
sending quantity: 80.0 (from factory 0 to customer 0)
sending quantity: 270.0 (from factory 1 to customer 1)
sending quantity: 230.0 (from factory 1 to customer 2)
sending quantity: 20.0 (from factory 2 to customer 2)
sending quantity: 160.0 (from factory 2 to customer 3)
sending quantity: 180.0 (from factory 2 to customer 4)

**********************
constants: slack, dual
Demand(0): -0.0, 4.0
Demand(1): -0.0, 5.0
Demand(2): -0.0, 4.0
Demand(3): -0.0, 3.0
Demand(4): -0.0, 4.0
Capacity(0): 420.0, 0.0
Capacity(1): -0.0, -1.0
Capacity(2): 140.0, 0.0


## 多品種

In [13]:
# 定数
d = [[80, 85, 300, 6], [270, 160, 400, 7], [250, 130, 350, 4],
     [160, 60, 200, 3], [180, 40, 150, 5]]  # 顧客iの品種kの需要
produce = [[1, 3], [0, 1, 2], [1, 2, 3]]  # 各工場jで生産可能な品種k
weight = [5, 2, 3, 4]  # 品種kの重量
M = [3000, 3000, 3000]  # 工場jの生産量の上限
cost = [[4, 6, 9], [5, 4, 7], [6, 3, 4], [8, 5, 3], [10, 8, 4]]  # 顧客iに工場jから輸送するときにかかるコスト
I = range(len(d))  # 顧客の集合
J = range(len(M))  # 工場の集合
K = range(len(weight))  # 品種の集合

model = Model('multi-commodity transportation')

# 品種ごとのコストを定義：単位コスト×重量
c = {} # 顧客iに工場jから品種kを輸送するときにかかるコスト
for i in I:
    for j in J:
        for k in produce[j]:
            c[i, j, k] = cost[i][j] * weight[k] 

# 変数
x = {}
for i, j, k in c:
    # 工場jが品種kを製造可能な場合のみx_ijkを定義
    x[i, j, k] = model.addVar(vtype='C', lb=0)
model.update()

# 定式化
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i, j, k] for j in J if (i, j, k) in x) == d[i][k])
for j in J:
    model.addConstr(quicksum(x[i, j, k] for (i, j2, k) in x if j2==j) <= M[j])
model.setObjective(quicksum(c[i, j, k] * x[i, j, k] for (i, j, k) in x), GRB.MINIMIZE)

# 最適化実行
model.optimize()
print('Optimal value: ', model.ObjVal)
eps = 1.0e-6
for i, j, k in x:
    if x[i, j, k].X > eps:
        print(f'sending quantity of commodity {k}: {x[i, j, k].X} (from factory {j} to customer {i})')

Optimal value:  43536.0
sending quantity of commodity 1: 85.0 (from factory 0 to customer 0)
sending quantity of commodity 3: 6.0 (from factory 0 to customer 0)
sending quantity of commodity 0: 80.0 (from factory 1 to customer 0)
sending quantity of commodity 2: 300.0 (from factory 1 to customer 0)
sending quantity of commodity 3: 7.0 (from factory 0 to customer 1)
sending quantity of commodity 0: 270.0 (from factory 1 to customer 1)
sending quantity of commodity 1: 160.0 (from factory 1 to customer 1)
sending quantity of commodity 2: 400.0 (from factory 1 to customer 1)
sending quantity of commodity 0: 250.0 (from factory 1 to customer 2)
sending quantity of commodity 1: 130.0 (from factory 1 to customer 2)
sending quantity of commodity 2: 350.0 (from factory 1 to customer 2)
sending quantity of commodity 3: 4.0 (from factory 2 to customer 2)
sending quantity of commodity 0: 160.0 (from factory 1 to customer 3)
sending quantity of commodity 1: 60.0 (from factory 2 to customer 3)
sendi

tuplelistを使って下のようにも定式化できる

In [19]:
arcs = tuplelist([(i, j, k) for (i, j, k) in x])

# 定式化
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i, j, k] for (i, j, k) in arcs.select(i, '*', k)) == d[i][k])
for j in J:
    model.addConstr(quicksum(x[i, j, k] for (i, j, k) in arcs.select('*', j, '*')) <= M[j])
model.setObjective(quicksum(c[i, j, k] * x[i, j, k] for (i, j, k) in x), GRB.MINIMIZE)

# 最適化実行
model.optimize()
print('Optimal value: ', model.ObjVal)

Optimal value:  43536.0


# 混合問題

In [18]:
## 定数 ##
# 原料iの成分kの含有率
a = [[0.25, 0.15, 0.3], [0.3, 0.3, 0.1],
     [0.15, 0.65, 0.05], [0.1, 0.05, 0.85]]
# 原料iのコスト
c = [5, 6, 8, 20]
# 成分kの上下限
UB = [0.2, 0.35, 1.0]
LB = [0.1, 0, 0.45]

I = range(len(c))
K = range(len(UB))

model = Model('product mix')

## 変数 ##
x = {}
for i in I:
    x[i] = model.addVar(vtype='C', lb=0)
model.update()

## 定式化 ##
model.addConstr(quicksum(x[i] for i in I) == 1)
for k in K:
    model.addConstr(LB[k] <= quicksum(a[i][k] * x[i] for i in I))
    model.addConstr(quicksum(a[i][k] * x[i] for i in I) <= UB[k])
model.setObjective(quicksum(c[i] * x[i] for i in I), GRB.MINIMIZE)

model.optimize()
for i in I:
    print(f'{i}: {x[i].X: .3f}')

0:  0.649
1:  0.000
2:  0.054
3:  0.297


# 分数最適化

In [21]:
LB, UB, EPS = 0.0, 1.0, 0.01

while True:
    model = Model('fractional2')
    x = model.addVar(vtype='I', lb=0)
    y = model.addVar(vtype='I', lb=0)
    z = model.addVar(vtype='I', lb=0)
    model.update()
    
    # 目的間数値がtheta以下の解があるか否か判定する部分問題
    theta = (LB + UB) / 2
    model.addConstr(x + y + z == 32)
    model.addConstr(2*x + 4*y + 8*z == 80)
    model.addConstr((2*theta - 1)*x +(4*theta - 1)*y >= 0)
    model.setObjective(1, GRB.MINIMIZE)
    model.optimize()
    
    if model.Status == 1:#GRB.OPTIMAL:
        UB = theta
        if UB - LB <= EPS:
            break
    else:
        LB = theta

print(f'(x, y, z) = {x.X, y.X, z.X}')

(x, y, z) = (24.0, 8.0, 0.0)


# 多制約0-1ナップサック問題

In [23]:
# 多制約0-1ナップサック問題を解く関数
def mkp(I, J, v, a, b):
    model = Model('mkp')
    
    ## 変数 ##
    x = {}
    for i in I:
        x[i] = model.addVar(vtype='B', name=f'x_{i}')
    model.update()
    
    ## 定式化 ##
    for j in J:
        model.addConstr(quicksum(a[i][j] * x[i] for i in I) <= b[j])
    model.setObjective(quicksum(v[i] * x[i] for i in I), GRB.MAXIMIZE)
    model.update()
    
    return model

In [35]:
## 定数 ##
# アイテムiの価値
v = [16, 19, 23, 28]
# アイテムiの制約jに対する重み
a = [[2, 3000], [3, 3500], [4, 5100], [5, 7200]]
# 制約jの上限値
b = [7, 10000]

I = range(len(v))
J = range(len(b))

## 最適化実行 ##
model = mkp(I, J, v, a, b)
model.optimize()
print('Optimal value: ', model.ObjVal)
for v in model.getVars():
    print(f'{v.VarName}: {v.X}')

Optimal value:  42.0
x_0: 0.0
x_1: 1.0
x_2: 1.0
x_3: 0.0


# 栄養問題

In [58]:
## 定数 ##
# 商品jが含んでいる栄養素iの量
n = [[556, 556, 356, 431, 249, 138, 69],
     [39, 46, 42, 45, 30, 10, 17],
     [30, 26, 14, 20, 3, 7, 1],
     [147, 97, 28, 9, 0, 80, 750],
     [10, 9, 1, 2, 5, 2, 2],
     [221, 142, 76, 37, 7, 227, 18],
     [2.4, 2.4, 0.7, 0.9, 0.6, 0, 0]]
# 商品jの価格
c = [360, 320, 270, 290, 190, 170, 100]
# 栄養素iの上下限
a = [3000, 375, 60, 750, 100, 900, 7.5]
b = [2000, 300, 50, 500, 85, 660, 6.0]

I = range(len(a))
J = range(len(c))

model = Model('diet problem')

## 変数 ##
x = {}
for j in J:
    x[j] = model.addVar(vtype='I', name=f'x_{j}', lb=0)

# 制約の逸脱に対するペナルティ
alpha = 10000
d, s = {}, {}
for i in I:
    d[i] = model.addVar(vtype='I', name=f'd_{i}', lb=0)
    s[i] = model.addVar(vtype='I', name=f's_{i}', lb=0)    
model.update()

## 定式化 ##
for i in I:
    model.addConstr(a[i] - d[i] <= quicksum(n[i][j] * x[j] for j in J))
    model.addConstr(quicksum(n[i][j] * x[j] for j in J) <= b[i] + s[i])
model.setObjective(quicksum(c[j] * x[j] for j in J) + alpha*quicksum(d[i] + s[i] for i in I), GRB.MINIMIZE)

model.optimize()

for v in model.getVars():
    if v.X > 1.0e-6:
        print(f'{v.VarName}: {v.X}')

d_0: 91.0
d_1: 109.0
d_3: 251.0
d_4: 53.0
s_0: 909.0
s_2: 66.0
s_5: 248.0
s_6: 4.0
x_0: 2.0
x_1: 1.0
x_2: 1.0
x_4: 3.0
x_5: 1.0
